# Imports

In [8]:
import pandas as pd
import numpy as np
import os
import pathlib

# Observer les données

In [5]:
data_raw = pd.read_csv("../data/prepared/project-1-at-2024-12-27-15-41-3fe0002a_prepared.csv", sep=";")
data_raw.head(1)

,inner_id,topic,topic_idx,data.text,data.name,file_upload,id,data.url,data.username,data.date
0,1,"[""social""]",[11],Stop #ViolenceAgainstWomen!\n\n#OrangeTheWorld...,L’Europe Ensemble,1de050d6-bluesky-comptes-politiques-2_.csv,1,https://bsky.app/profile/fabiennekeller.bsky.s...,ensembleue.bsky.social,2024-11-25 19:25:10.459000


In [6]:
dict_labels_index = {'autre':0,
    'culture et médias':1,
    'environnement':2,
    'économie':3,
    'éducation':4,
    'fiscalité':5,
    'institutions':6,
    'immigration':7,
    'international':8,
    'outre-mer':9,
    'pouvoir d’achat':10,
    'social':11,
    'services publics':12,
    'santé':13,
    'sécurité / justice':14,
    'vie politique':15}

dict_index_labels = {value:key for key, value in dict_labels_index.items()}

In [23]:
labels_arr = np.zeros((data_raw.shape[0], 16))
labels_raw = data_raw['topic_idx'].tolist()
labels_ints = []
for labs in labels_raw:
    labels_ints.append([int(value) for value in labs.replace('[', '').replace(']', '').split(',')])

for i in range(labels_arr.shape[0]):
    labs = labels_ints[i]
    for value in labs:
        labels_arr[i, value] = 1
labels_list = labels_arr.tolist()
df_labels = pd.DataFrame(labels_list, columns=dict_labels_index.keys())

df_data = pd.concat((data_raw.loc[:, 'data.text'], df_labels), axis=1)
df_data.head()

,data.text,autre,culture et médias,environnement,économie,éducation,fiscalité,institutions,immigration,international,outre-mer,pouvoir d’achat,social,services publics,santé,sécurité / justice,vie politique
0,Stop #ViolenceAgainstWomen!\n\n#OrangeTheWorld...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,La violence à l’égard des femmes reste une urg...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,"« Notre position sur le Mercosur, depuis 2019,...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Nouvelle session plénière à Strasbourg ! À l’o...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Désaveu cinglant pour Meloni qui se résout à r...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_data['nlabels'] = df_data.iloc[:, 1:].sum(axis=1)
df_data.head()

,data.text,autre,culture et médias,environnement,économie,éducation,fiscalité,institutions,immigration,international,outre-mer,pouvoir d’achat,social,services publics,santé,sécurité / justice,vie politique,nlabels
0,Stop #ViolenceAgainstWomen!\n\n#OrangeTheWorld...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,La violence à l’égard des femmes reste une urg...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,"« Notre position sur le Mercosur, depuis 2019,...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Nouvelle session plénière à Strasbourg ! À l’o...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
4,Désaveu cinglant pour Meloni qui se résout à r...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [25]:
d_nlabels = {col: 0 for col in df_data.columns}

In [30]:
for col in df_data.columns:
    d_nlabels[col] = df_data.loc[df_data[col] == 1, 'nlabels'].mean()

for k,v in d_nlabels.items():
    print(f"{k} : {v:.2f}")   

data.text : nan
autre : 1.00
culture et médias : 1.53
environnement : 1.76
économie : 2.04
éducation : 1.79
fiscalité : 1.72
institutions : 2.03
immigration : 1.50
international : 1.59
outre-mer : 1.37
pouvoir d’achat : 1.72
social : 1.67
services publics : 2.07
santé : 2.17
sécurité / justice : 1.72
vie politique : 1.41
nlabels : 1.00


# Data

In [13]:
dir_data = '../data'
path = pathlib.Path(dir_data)
for p in path.rglob("*"):
    print(p)

..\data\collecte du 26 decembre
..\data\collecte du 26 novembre
..\data\infos collecte données communalytics.txt
..\data\project-1-at-2024-12-23-15-32-de79597b.csv
..\data\project-1-at-2024-12-23-15-44-7d7b1a28.json
..\data\project-1-at-2024-12-26-10-59-82fcc174.json
..\data\collecte du 26 decembre\bluesky-comptes-politiques-1-new_-only-new.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-1-new_-only-new2.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-1-new_.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-1-new_.xlsx
..\data\collecte du 26 decembre\bluesky-comptes-politiques-2-new_-only-new.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-2-new_.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-2-new_.xlsx
..\data\collecte du 26 decembre\bluesky-comptes-politiques-3-new_-only-new.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-3-new_.csv
..\data\collecte du 26 decembre\bluesky-comptes-politiques-3-ne

In [27]:
df_pol_1 = pd.read_csv("../data/collecte du 26 novembre/bluesky-comptes-politiques_.csv")
df_pol_1['date'] = pd.to_datetime(df_pol_1['date'], format="ISO8601")
df_pol_1.sort_values(by='date', ascending=False, inplace=True)
datetime_first_collect_df_pol_1 = df_pol_1.date.iloc[0]
datetime_first_collect_df_pol_1

Timestamp('2024-11-25 19:31:55.840000')

In [28]:
df_pol_2 = pd.read_csv("../data/collecte du 26 novembre/bluesky-comptes-politiques-2_.csv")
df_pol_2['date'] = pd.to_datetime(df_pol_2['date'], format="ISO8601")
df_pol_2.sort_values(by='date', ascending=False, inplace=True)
datetime_first_collect_df_pol_2 = df_pol_2.date.iloc[0]
datetime_first_collect_df_pol_2

Timestamp('2024-11-26 08:59:51.052000')

In [29]:
df_pol_3 = pd.read_csv("../data/collecte du 26 novembre/bluesky-comptes-politiques-3_.csv")
df_pol_3['date'] = pd.to_datetime(df_pol_3['date'], format="ISO8601")
df_pol_3.sort_values(by='date', ascending=False, inplace=True)
datetime_first_collect_df_pol_3 = df_pol_3.date.iloc[0]
datetime_first_collect_df_pol_3

Timestamp('2024-11-26 08:47:08.359000')

In [36]:
df_pol_new_1 = pd.read_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-1-new_.csv")
df_pol_new_1['date'] = pd.to_datetime(df_pol_new_1['date'], format="ISO8601")
df_pol_new_1.sort_values(by='date', ascending=False, inplace=True)
df_pol_1_only_new = df_pol_new_1[df_pol_new_1.date >= datetime_first_collect_df_pol_1]
print(df_pol_1_only_new.shape)
df_pol_1_only_new.to_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-1-only-new_.csv", sep=',', index=False)


(203, 15)


In [37]:
df_pol_new_2 = pd.read_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-2-new_.csv")
df_pol_new_2['date'] = pd.to_datetime(df_pol_new_2['date'], format="ISO8601")
df_pol_new_2.sort_values(by='date', ascending=False, inplace=True)
df_pol_2_only_new = df_pol_new_2[df_pol_new_2.date >= datetime_first_collect_df_pol_2]
print(df_pol_2_only_new.shape)
df_pol_2_only_new.to_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-2-only-new_.csv", sep=',', index=False)


(100, 15)


In [39]:
df_pol_new_3 = pd.read_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-3-new_.csv")
df_pol_new_3['date'] = pd.to_datetime(df_pol_new_3['date'], format="ISO8601")
df_pol_new_3.sort_values(by='date', ascending=False, inplace=True)
df_pol_3_only_new = df_pol_new_3[df_pol_new_3.date >= datetime_first_collect_df_pol_3]
print(df_pol_3_only_new.shape)
df_pol_3_only_new.to_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-3-only-new_.csv", sep=',', index=False)


(169, 15)


In [71]:
df_renaissance = pd.read_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-renaissance_.csv")
df_renaissance['date'] = pd.to_datetime(df_renaissance['date'], format="ISO8601")
df_renaissance.sort_values(by='date', ascending=True, inplace=True)
df_renaissance.dropna(subset=['text'], inplace=True) # Pas de texte vide
df_renaissance = df_renaissance[df_renaissance.loc[:, 'text'].str.len() >= 30] # Posts d'au moins 20 caractères
df_renaissance.reset_index(drop=True)
print(df_renaissance.shape)
df_renaissance_sampled = df_renaissance.sample(200, random_state=42).copy() # 42 pour répétabilité

with open("../data/collecte du 26 decembre/index_renaissance.txt", "w") as file:
    file.write("used_index\n")
    indeces = df_renaissance_sampled.index.to_list()
    for index in indeces:
        file.write(str(index))
        file.write('\n')

df_renaissance_sampled.to_csv("../data/collecte du 26 decembre/bluesky-comptes-politiques-renaissance-sampled_.csv", sep=',', index=False)


(2074, 15)
